# Assignment - Basic Pandas
<sup>Created by Natawut Nupairoj, Department of Computer Engineering, Chulalongkorn University</sup>

Using pandas to explore youtube trending data from GB (GBvideos.csv and GB_category_id.json) and answer the questions.

In [38]:
import pandas as pd
import numpy as np

To simplify data retrieval process on Colab, we heck if we are in the Colab environment and download data files from a shared drive and save them in folder "data".

For those using jupyter notebook on the local computer, you can read data directly assuming you save data in the folder "data".

In [39]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !wget https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/datasets/data.tgz -O data.tgz
    !tar -xzvf data.tgz

--2024-01-10 13:47:59--  https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/datasets/data.tgz
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kaopanboonyuen/2110446_DataScience_2021s2/main/datasets/data.tgz [following]
--2024-01-10 13:47:59--  https://raw.githubusercontent.com/kaopanboonyuen/2110446_DataScience_2021s2/main/datasets/data.tgz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45477462 (43M) [application/octet-stream]
Saving to: ‘data.tgz’

data.tgz            100%[===================>]  43.37M  --.-KB/s    in 0.1s    

2024-01-10 13:47:59 (364 MB/s) - ‘data.tgz’ sa

## How many rows are there in the GBvideos.csv after removing duplications?

In [40]:
gb_vdo = pd.read_csv('data/GBvideos.csv') #read file
gb_vdo.drop_duplicates(inplace=True) #remove duplicate
len(gb_vdo) #how many row

38745

## How many VDO's that contain at least one record (row) of "dislikes" more than "likes"?  <font color=red>DO NOT group by the data and make sure that you count only unique title!</font>

In [41]:
gb_tmp = gb_vdo[gb_vdo.dislikes > gb_vdo.likes] #check that what vdo that dislikes more than likes
len(gb_tmp["title"].unique()) #how many row that check unique of title too

56

## How many VDO that are trending on 22 Jan 2018 with comments more than 10,000 comments?

In [42]:
from datetime import datetime, timezone #import datetime

In [43]:
time = datetime(2018, 1, 22, tzinfo=timezone.utc) #set datetime 22 Jan 2018
#check that trending_date on 22 Jan 2018 and comment more than 10000 or not
condition_list = (pd.to_datetime(gb_vdo.trending_date, format='%y.%d.%m', errors='ignore', utc=True) == time) & (gb_vdo["comment_count"] > 10000)
len(gb_vdo[condition_list]) #how many row that have true condition

28

## Which date that has the minimum average number of comments per VDO?

In [54]:
#calculate mean of comment that group by trending_date and reset index
gb_tmp = gb_vdo.groupby("trending_date")["comment_count"].mean().reset_index(name="avg_comment")
gb_tmp["trending_date"][gb_tmp["avg_comment"] == gb_tmp["avg_comment"].min()] #display date that have min avg comment_count

15    17.15.11
Name: trending_date, dtype: object

## Compare "Sports" and "Comady", how many days that there are more total daily views of VDO in "Sports" category than in "Comady" category?

In [46]:
import json #import json

In [47]:
with open("data/GB_category_id.json") as fd:
    cate = json.load(fd) #load json

In [48]:
cate_list = [] #create list
for d in cate["items"]:
    cate_list.append((int(d["id"]), d["snippet"]["title"])) #input id and title to list

In [49]:
cate_df = pd.DataFrame(cate_list, columns=["id", "category"]) #create dateframe with cate_list

In [50]:
#merge dateframe that create before to gb_vdo that link from category_id and id
gb_vdo_withcate = gb_vdo.merge(cate_df, left_on="category_id", right_on="id")
#get row that category is sport or comedy
gb_vdo_with_sport_comedy = gb_vdo_withcate[(gb_vdo_withcate["category"] == "Sports") | (gb_vdo_withcate["category"] == "Comedy")]

In [51]:
#groupby to get total view from each category and trending_date
gb_vdo_group = gb_vdo_with_sport_comedy.groupby(["trending_date","category"])["views"].sum().reset_index(name="total_views")
gb_vdo_group

,trending_date,category,total_views
0,17.01.12,Comedy,10889970
1,17.01.12,Sports,9879808
2,17.02.12,Comedy,13302391
3,17.02.12,Sports,10856461
4,17.03.12,Comedy,14259944
...,...,...,...
405,18.31.01,Sports,8431519
406,18.31.03,Comedy,21528161
407,18.31.03,Sports,22546308
408,18.31.05,Comedy,16743398


In [52]:
gb_vdo_sport = gb_vdo_group[gb_vdo_group["category"] == "Sports"] #get dataframe that category is sport
gb_vdo_comedy = gb_vdo_group[gb_vdo_group["category"] == "Comedy"] #get dataframe that category is comedy
#merge two dateframe (sport dataframe and comedy dataframe with rename column)
gb_vdo_compare = pd.merge(gb_vdo_sport, gb_vdo_comedy, on="trending_date", suffixes=("_sports", "_comedy"))
gb_vdo_compare

,trending_date,category_sports,total_views_sports,category_comedy,total_views_comedy
0,17.01.12,Sports,9879808,Comedy,10889970
1,17.02.12,Sports,10856461,Comedy,13302391
2,17.03.12,Sports,10875249,Comedy,14259944
3,17.04.12,Sports,11399173,Comedy,16617431
4,17.05.12,Sports,11261410,Comedy,19089597
...,...,...,...,...,...
200,18.30.04,Sports,15114095,Comedy,17802688
201,18.30.05,Sports,5153521,Comedy,16427316
202,18.31.01,Sports,8431519,Comedy,14840766
203,18.31.03,Sports,22546308,Comedy,21528161


## *Final Answer*

---



---



In [53]:
#check which trending_date that have total views of sport more than comedy
count_more_views = (gb_vdo_compare[gb_vdo_compare["total_views_sports"] > gb_vdo_compare["total_views_comedy"]])
len(count_more_views) #how many row that is answer

49